# <font color = 'blue' > Dermnet Classification
## <font color= 'blue'> Creating a custom object detection model with YOLO

#<font color = "blue"> Imports

In [1]:
from IPython import display

In [2]:
!pip install matplotlib
!pip install roboflow
!pip install ultralytics
!pip install roboflow
!pip install tensorflow
!pip install torch
display.clear_output()

#<font color = "blue"> GPU Check

In [4]:
!nvidia-smi # making sure GPU is running

Sun Oct 27 14:51:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
# another check to make sure GPU is working
import tensorflow as tf
from tensorflow.python.client import device_lib

numGPUs = len(tf.config.experimental.list_physical_devices('GPU'))
print('Num GPUs Available: ', numGPUs)
if numGPUs > 0:
  print(tf.test.gpu_device_name())
  print(device_lib.list_local_devices()[1].physical_device_desc)

# check if cuda is available
import torch
if torch.cuda.is_available():
  print("CUDA is available!")
else:
  print("CUDA is not available. Using CPU.")

Num GPUs Available:  1
/device:GPU:0
device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
CUDA is available!


# <font color = 'blue'> Load Dermnet Dataset
- manually add kaggle.json file (contains api key) to kaggle folder
- use kagglehub to download dataset

In [6]:
# MANUALLY ADD KAGGLE.JSON FILE TO KAGGLE FOLDER

import os
import kagglehub


kaggle_folder = "/content/kaggle"
os.makedirs(kaggle_folder)

# change current working directory to kaggle folder (kagglehub only downloads in current w directory)
os.chdir(kaggle_folder)

# download latest version
!kaggle datasets download -d shubhamgoel27/dermnet

print("Current working directory:", os.getcwd())  # kaggle folder path
print("Downloaded files:", os.listdir())  # downloaded data files

Dataset URL: https://www.kaggle.com/datasets/shubhamgoel27/dermnet
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
100% 1.72G/1.72G [00:24<00:00, 99.2MB/s]
100% 1.72G/1.72G [00:24<00:00, 77.0MB/s]
Current working directory: /content/kaggle
Downloaded files: ['dermnet.zip']


In [3]:
# unzip dataset file
#dataset_directory = "/content/kaggle/dermnet"
!unzip dermnet.zip -d dermnet
display.clear_output()

In [7]:
# delete classes from train and test folder
import shutil
train1_delete = "/content/kaggle/dermnet/train/Tinea Ringworm Candidiasis and other Fungal Infections"
train2_delete = "/content/kaggle/dermnet/train/Herpes HPV and other STDs Photos"
train3_delete = "/content/kaggle/dermnet/train/Bullous Disease Photos"
test1_delete = "/content/kaggle/dermnet/test/Tinea Ringworm Candidiasis and other Fungal Infections"
test2_delete = "/content/kaggle/dermnet/test/Herpes HPV and other STDs Photos"
test3_delete = "/content/kaggle/dermnet/test/Bullous Disease Photos"

delete_classes = [train1_delete, train2_delete, train3_delete, test1_delete, test2_delete, test3_delete]

for folder_path in delete_classes:
# Check if the folder exists

  if os.path.exists(folder_path):
      # Delete the folder and its contents
      shutil.rmtree(folder_path)
      print(f"Folder '{folder_path}' and its contents have been deleted.")
  else:
      print(f"Folder '{folder_path}' does not exist.")

Folder '/content/kaggle/dermnet/train/Tinea Ringworm Candidiasis and other Fungal Infections' does not exist.
Folder '/content/kaggle/dermnet/train/Herpes HPV and other STDs Photos' does not exist.
Folder '/content/kaggle/dermnet/train/Bullous Disease Photos' does not exist.
Folder '/content/kaggle/dermnet/test/Tinea Ringworm Candidiasis and other Fungal Infections' does not exist.
Folder '/content/kaggle/dermnet/test/Herpes HPV and other STDs Photos' does not exist.
Folder '/content/kaggle/dermnet/test/Bullous Disease Photos' does not exist.


In [ ]:
import os

def delete_name_part(directory):
    """
    Parses through folder names in a directory and removes "photos" from their names.

    Args:
        directory: The path to the directory containing the folders.
    """
    for filename in os.listdir(directory):
        if os.path.isdir(os.path.join(directory, filename)):  # check if directory exists
            if "Photos" in filename:
                new_filename = filename.replace("Photos", "")
                new_filename = new_filename.strip()  # remove leading/trailing spaces
                os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
                print(f"Renamed '{filename}' to '{new_filename}'")

# Example usage
train_directory_path = "/content/kaggle/dermnet/train"
test_directory_path = "/content/kaggle/dermnet/test"
delete_name_part(train_directory_path)
delete_name_part(test_directory_path)

Renamed 'Poison Ivy Photos and other Contact Dermatitis' to 'Poison Ivy  and other Contact Dermatitis'
Renamed 'Eczema Photos' to 'Eczema'
Renamed 'Hair Loss Photos Alopecia and other Hair Diseases' to 'Hair Loss  Alopecia and other Hair Diseases'
Renamed 'Vasculitis Photos' to 'Vasculitis'
Renamed 'Acne and Rosacea Photos' to 'Acne and Rosacea'
Renamed 'Atopic Dermatitis Photos' to 'Atopic Dermatitis'
Renamed 'Poison Ivy Photos and other Contact Dermatitis' to 'Poison Ivy  and other Contact Dermatitis'
Renamed 'Eczema Photos' to 'Eczema'
Renamed 'Hair Loss Photos Alopecia and other Hair Diseases' to 'Hair Loss  Alopecia and other Hair Diseases'
Renamed 'Vasculitis Photos' to 'Vasculitis'
Renamed 'Acne and Rosacea Photos' to 'Acne and Rosacea'
Renamed 'Atopic Dermatitis Photos' to 'Atopic Dermatitis'


In [ ]:
# print class names for .yaml file:
i = 0

for filename in os.listdir(train_directory_path):

  if os.path.isdir(os.path.join(train_directory_path, filename)):  # check if directory exists
    print(str(i) + ": ", filename)
    i += 1

0:  Scabies Lyme Disease and other Infestations and Bites
1:  Vasculitis
2:  Eczema
3:  Poison Ivy  and other Contact Dermatitis
4:  Warts Molluscum and other Viral Infections
5:  Melanoma Skin Cancer Nevi and Moles
6:  Lupus and other Connective Tissue diseases
7:  Hair Loss  Alopecia and other Hair Diseases
8:  Psoriasis pictures Lichen Planus and related diseases
9:  Vascular Tumors
10:  Light Diseases and Disorders of Pigmentation
11:  Cellulitis Impetigo and other Bacterial Infections
12:  Seborrheic Keratoses and other Benign Tumors
13:  Atopic Dermatitis
14:  Urticaria Hives
15:  Acne and Rosacea
16:  Exanthems and Drug Eruptions
17:  Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions
18:  Nail Fungus and other Nail Disease
19:  Systemic Disease


In [ ]:
# before starting to train model, need to change directory again (back to content folder)
# .yaml file is assuming current directory is in content folder --> in .yaml file, referenced val images as test directory
# (YAML FILE NOT REQUIRED)

# change current working directory to content folder
content_folder = "/content"
os.chdir(content_folder)
print("Current working directory:", os.getcwd())

Current working directory: /content


# <font color = "blue"> Set up Model

In [ ]:
from roboflow import Roboflow
import ultralytics

In [ ]:
dataset_config_path = '/content/kaggle/dermnet' # DO NOT HAVE TO USE A .YAML FILE

# <font color = "blue"> Benchmark YOLO
- 25 imag

#### *metrics*
**Average Precision (AP)**
- computes the area under the precision-recall curve
- providing a single value that encapsulates model's precision and recall performance

**Mean Average Precision (mAP)**
- mAP extends the concept of AP by calculating the average AP values across multiple object classes
- useful in multi-class object detection scenarios
- provides comprehensive evaluation of model performance

**Precision and Recall**
- precision quantifies proportion of true positives among all positive predictions, assessing the model's capability to avoid false positives. On the other hand, Recall calculates the proportion of true positives among all actual positives, measuring the model's ability to detect all instances of a class.

**F1 Score**
- harmonic mean of precision and recall
- balanced assessment of a model's performance while considering both false positives and false negatives

In [ ]:
model.names

{0: 'Acne and Rosacea',
 1: 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
 2: 'Atopic Dermatitis',
 3: 'Cellulitis Impetigo and other Bacterial Infections',
 4: 'Eczema',
 5: 'Exanthems and Drug Eruptions',
 6: 'Hair Loss  Alopecia and other Hair Diseases',
 7: 'Light Diseases and Disorders of Pigmentation',
 8: 'Lupus and other Connective Tissue diseases',
 9: 'Melanoma Skin Cancer Nevi and Moles',
 10: 'Nail Fungus and other Nail Disease',
 11: 'Poison Ivy  and other Contact Dermatitis',
 12: 'Psoriasis pictures Lichen Planus and related diseases',
 13: 'Scabies Lyme Disease and other Infestations and Bites',
 14: 'Seborrheic Keratoses and other Benign Tumors',
 15: 'Systemic Disease',
 16: 'Urticaria Hives',
 17: 'Vascular Tumors',
 18: 'Vasculitis',
 19: 'Warts Molluscum and other Viral Infections'}

#### <font color = 'purple'> train model

In [ ]:
from ultralytics import YOLO

dataset_config_path = '/content/kaggle/dermnet' # DO NOT HAVE TO USE A .YAML FILE
model = YOLO('yolov8n-cls.pt') # load a pretrained model
model.train(data=dataset_config_path, epochs=10, batch=25) # default images per batch is 16

100%|██████████| 5.31M/5.31M [00:00<00:00, 61.2MB/s]


Ultralytics 8.3.23 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/kaggle/dermnet, epochs=10, time=None, patience=100, batch=25, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_bo

100%|██████████| 5.35M/5.35M [00:00<00:00, 317MB/s]


AMP: checks passed ✅


train: Scanning /content/kaggle/dermnet/train... 13404 images, 0 corrupt: 100%|██████████| 13404/13404 [00:03<00:00, 3643.76it/s]


...

                   all       0.58      0.872



10 epochs completed in 0.452 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train/weights/best.pt, 3.0MB

Validating runs/classify/train/weights/best.pt...
Ultralytics 8.3.23 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-cls summary (fused): 73 layers, 1,460,500 parameters, 0 gradients, 3.3 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /content/kaggle/dermnet/train... found 13404 images in 20 classes ✅ 
val: None...
test: /content/kaggle/dermnet/test... found 3462 images in 20 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 70/70 [00:25<00:00,  2.70it/s]


                   all      0.579      0.873
Speed: 0.1ms preprocess, 0.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a99eea1c1c0>
curves: []
curves_results: []
fitness: 0.7258810102939606
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.5791450142860413, 'metrics/accuracy_top5': 0.8726170063018799, 'fitness': 0.7258810102939606}
save_dir: PosixPath('runs/classify/train')
speed: {'preprocess': 0.07188519875070802, 'inference': 0.26055452522000155, 'loss': 0.0003240894954511707, 'postprocess': 0.00028387099452820353}
task: 'classify'
top1: 0.5791450142860413
top5: 0.8726170063018799

#### <font color = 'purple'> download training results

In [ ]:
# download training results


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!zip -r /content/runs.zip /content/runs

  adding: content/runs/ (stored 0%)
  adding: content/runs/classify/ (stored 0%)
  adding: content/runs/classify/train/ (stored 0%)
  adding: content/runs/classify/train/results.png (deflated 8%)
  adding: content/runs/classify/train/results.csv (deflated 58%)
  adding: content/runs/classify/train/confusion_matrix_normalized.png (deflated 10%)
  adding: content/runs/classify/train/val_batch2_labels.jpg (deflated 2%)
  adding: content/runs/classify/train/val_batch0_pred.jpg (deflated 1%)
  adding: content/runs/classify/train/val_batch2_pred.jpg (deflated 2%)
  adding: content/runs/classify/train/val_batch1_labels.jpg (deflated 1%)
  adding: content/runs/classify/train/train_batch2.jpg (deflated 4%)
  adding: content/runs/classify/train/val_batch1_pred.jpg (deflated 1%)
  adding: content/runs/classify/train/train_batch1.jpg (deflated 4%)
  adding: content/runs/classify/train/events.out.tfevents.1729990135.d67633aadd39.458.0 (deflated 93%)
  adding: content/runs/classify/train/val_batch0_

In [ ]:
from google.colab import files
files.download('runs.zip')
files.download('yolo11n.pt')
files.download('yolov8n-cls.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### <font color = 'purple'> view training results
(receiving error when retrieving; but results were saved in file locally)
**maybe try renaming test folder to val folder**

In [ ]:
validation_results = model.val()

Ultralytics 8.3.23 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /content/kaggle/dermnet/train... found 13404 images in 20 classes ✅ 
val: None...
test: /content/kaggle/dermnet/test... found 3462 images in 20 classes ✅ 


TypeError: expected str, bytes or os.PathLike object, not NoneType

#### <font color = 'purple'> run validation

In [ ]:
#val_metrics = model.val(save_json=True, name="val_1")  # no arguments needed, dataset and settings remembered
#print(val_metrics.box.map)  # map
#val_metrics.box.maps  # a list contains map50-95 of each category

# <font color = 'blue'> Predict

In [ ]:

# Run batched inference on a list of images
results = model(["nailFung1.jpg"])  # return a list of Results objects
print("PLEASE = ", results[0].probs.top1)
# Process results list
#for result in results:
    #boxes = result.boxes  # Boxes object for bounding box outputs
    #masks = result.masks  # Masks object for segmentation masks outputs
   # keypoints = result.keypoints  # Keypoints object for pose outputs
   # probs = result.probs  # Probs object for classification outputs
  #  obb = result.obb  # Oriented boxes object for OBB outputs
   # result.show()  # display to screen
  #  result.save(filename="result.jpg")  # save to disk

#pictureClass = model("nailFung1.jpg")



  # im1 predictions (pandas)


0: 224x224 Nail Fungus and other Nail Disease 1.00, Lupus and other Connective Tissue diseases 0.00, Psoriasis pictures Lichen Planus and related diseases 0.00, Warts Molluscum and other Viral Infections 0.00, Vascular Tumors 0.00, 4.1ms
Speed: 6.8ms preprocess, 4.1ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)
PLEASE =  10


In [ ]:
print(pictureClass)

[ultralytics.engine.results.Results object with attributes:

boxes: None
keypoints: None
masks: None
names: {0: 'Acne and Rosacea', 1: 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions', 2: 'Atopic Dermatitis', 3: 'Cellulitis Impetigo and other Bacterial Infections', 4: 'Eczema', 5: 'Exanthems and Drug Eruptions', 6: 'Hair Loss  Alopecia and other Hair Diseases', 7: 'Light Diseases and Disorders of Pigmentation', 8: 'Lupus and other Connective Tissue diseases', 9: 'Melanoma Skin Cancer Nevi and Moles', 10: 'Nail Fungus and other Nail Disease', 11: 'Poison Ivy  and other Contact Dermatitis', 12: 'Psoriasis pictures Lichen Planus and related diseases', 13: 'Scabies Lyme Disease and other Infestations and Bites', 14: 'Seborrheic Keratoses and other Benign Tumors', 15: 'Systemic Disease', 16: 'Urticaria Hives', 17: 'Vascular Tumors', 18: 'Vasculitis', 19: 'Warts Molluscum and other Viral Infections'}
obb: None
orig_img: array([[[ 61,  77, 106],
        [ 61,  77, 106],
  